# Introduction to Big Data

## Semester End Exam

####  Date   : March 2022
#### Duration : 

### INSTRUCTIONS:

1.	Candidates should answer all the questions in the same order provided in the question paper.
2.	Any activity that compromises the integrity of the examination will not be permitted.
3.	Students should complete the examination within the provided timeline.
4.  Once completed use <b> File> Dowload as > HTML(.html) </b> to download the notebook as html page into local drive. you must specifying path and check that its downloaded propery and shows output of each cell without error .
4.	Candidates are expected to check and ensure that the correct answer file (.html format) is uploaded in LMS.


### DATA DESCRIPTION: 


This dataset is real-state data of bangalore. This file has 9 attributes. Our main objective is to  process the data set using spark libraries to check the programing & conceptural understanding .

### ATTRIBUTES:

Dataset contains a total of 9 attributes as shown below 

- area_type :  Type of measurment area

- availability: Availabily status/date 

- location: locality of bengaluru 

- size: Number of bedrooms/ bhk

- society: Name of Socity

- total_sqft:  house Area  in sq-feet  

- bath: Number of bathrooms  

- balcony: Number of balaconies

- price: Price in lakh 


In [1]:
# import statements
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import isnull, when, count, col,avg
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import FMRegressor
from pyspark.ml import Pipeline
 
sc = SparkContext.getOrCreate()
spark = SparkSession(sc) 


In [19]:
from pyspark.sql.functions import min, max, avg

In [2]:
print(spark.version)

3.3.2


## Section C

Bangalore Housing Dataset is provided and loaded as Spark-DataFrame.  Using Spark libraries execute the steps, as questioned below.

In [3]:
# update this path for docker image
file_path='Bengaluru_House_Data.csv' 

#file_path='Bengaluru_House_Data.csv' 

#load the data spark Dataframe 
#df=spark.read.format("csv").option("header", "True").option("inferschema","True").load(file_path)

In [5]:
df=spark.read.format("csv").option("header","True").option("inferschema","True").load(file_path)

### 3.a.  Using PySpark and Spark-SQL libraries process the dataset to find out solutions of queries mentioned below.

### 3.a.(i).	How many total numbers of housing-properties are listed in the dataset? (2 marks)

In [9]:
df.count()
# Ans : 13320

13320

In [10]:
df.count()

13320

### 3.a.(ii). What is the average price of ‘3 BHK’ size housing-properties in ‘Electronic City’ location?( 3 marks)

In [5]:
df.filter( (df.location == 'Electronic City') & (df.size == '3 BHK') ).count()
# Ans:76

76

In [15]:
df.filter((df.location=='Electronic City')&(df.size=='3 BHK')).select(avg('price').alias('avg_price')).show()

+-----------------+
|        avg_price|
+-----------------+
|67.22092105263157|
+-----------------+



In [21]:
df.select(
    min("price").alias("Min_Price"),
    max("price").alias("Max_Price"),
    avg("price").alias("Avg_Price")
).show()

+---------+---------+------------------+
|Min_Price|Max_Price|         Avg_Price|
+---------+---------+------------------+
|      8.0|   3600.0|112.56562650150138|
+---------+---------+------------------+



### 3.a.(iii).   Which locality has maximum availability of  ‘Ready To Move’ housing-properties?  ( 5 marks)

In [6]:
df.filter(df.availability == 'Ready To Move').groupBy('location').count().orderBy(['count'],ascending = [False]).show(1)
# Ans: Whitefield

+----------+-----+
|  location|count|
+----------+-----+
|Whitefield|  331|
+----------+-----+
only showing top 1 row



In [28]:
df.filter(df.availability=='Ready To Move').groupby("location").count().orderBy('count', ascending=[False]).show(1)

+----------+-----+
|  location|count|
+----------+-----+
|Whitefield|  331|
+----------+-----+
only showing top 1 row



### 3.b. Using Spark ML libraries process the Dataframe as questioned below (in order to build a PySpark  regression ML-model on provided Bangalore Housing data)


#### 3.b.(i).  i.	For features, having more than one third of their entries as missing/null, replace null cells with the new value as ‘missing’.  For the remaining missing/null values - remove the corresponding row entry from the DataFrame. (3 marks)  

In [7]:
# fetch column wise mssing count
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

+---------+------------+--------+----+-------+----------+----+-------+-----+
|area_type|availability|location|size|society|total_sqft|bath|balcony|price|
+---------+------------+--------+----+-------+----------+----+-------+-----+
|        0|           0|       1|  16|   5502|         0|  73|    609|    0|
+---------+------------+--------+----+-------+----------+----+-------+-----+



In [34]:
df.select([count(when(isnull(c),c)).alias(c) for c in df.columns]).show()

+---------+------------+--------+----+-------+----------+----+-------+-----+
|area_type|availability|location|size|society|total_sqft|bath|balcony|price|
+---------+------------+--------+----+-------+----------+----+-------+-----+
|        0|           0|       1|  16|   5502|         0|  73|    609|    0|
+---------+------------+--------+----+-------+----------+----+-------+-----+



In [8]:
# repalce society column null entries with a new value as 'missing'
df = df.na.fill("missing",'society');


In [9]:
# removing the corresponding row entry with rest of the null entries
df = df.na.drop()

### 3.b.(ii). Convert all string columns into numeric values using StringIndexer transformer and make sure now DataFrame does not have any string columns anymore.(3 marks) 


In [10]:
from pyspark.ml import Pipeline
indexers = [StringIndexer(inputCol=column,
                          outputCol=column+"_indexex").fit(df) for column in ['area_type',
                                                                            'availability','location',
                                                                            'size','society','total_sqft']]
pipeline = Pipeline(stages=indexers)
df = pipeline.fit(df).transform(df)

In [11]:
## alternate code for doing StringIndexer transformation
# indexer_model = StringIndexer(inputCols = ["area_type", "availability", "location", "size", "total_sqft"],                         
#                          outputCols =["area_type_indexed", "availability_indexed",
#                                       "location_indexed","size_indexed","total_sqft_indexed"]).fit(df)
# df_indexed = indexer_model.transform(df)
# df_indexed = df_indexed.drop("area_type", "availability", "location",'size',"total_sqft")
# df_indexed.printSchema()

In [12]:
# drop the string columns 
df = df.drop('area_type','availability','location', 'size','society','total_sqft')

### 3.b.(iii). Using vectorAssembler combines all columns (except target column i.e. 'price') of spark DataFrame into single column (named as features). Make sure DataFrame now contains only two columns features and price.(3 marks)

In [13]:
features_col = df.columns
features_col.remove('price')

# Create the VectorAssembler object
assembler = VectorAssembler(inputCols= features_col, outputCol= "features")
df_assembled = assembler.transform(df)


df= df_assembled.drop('bath',
 'balcony',
 'location_index',
 'size_index',
 'total_sqft_index',
 'society_index','availability_index',
 'area_type_index')

df.columns

['price',
 'area_type_indexex',
 'availability_indexex',
 'location_indexex',
 'size_indexex',
 'society_indexex',
 'total_sqft_indexex',
 'features']

### 3.b.(iv) Split the vectorized dataframe into training and test sets with one fourth records being held for testing. (1 marks)

In [14]:
#split 
df_train , df_test = df.randomSplit([0.75,0.25], seed = 2022)

### 3.b.(v).	Train default LinearRegression model with features as 'featuresCol'   and ‘price’ as label and measure RMSE value on test set (5 marks)

In [15]:
lr = LinearRegression(featuresCol="features", labelCol="price")
lr_model = lr.fit(df_train)
lr_predicton  = lr_model.transform(df_test)
#df_predicton.show()
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(lr_predicton)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 120.219


### 3.b.(vi).	Train default Factorization machines regressor model with features as 'featuresCol'   and ‘price’ as label and measure RMSE value on test set.  Which among LinearRegression and Factorization machines regressor model performs better? (marks 4 +1)

In [16]:
#Factorization machines regressor
from pyspark.ml.regression import FMRegressor
fm =  FMRegressor(featuresCol="features", labelCol="price")
fm_model = fm.fit(df_train)
fm_predicton  = fm_model.transform(df_test)
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(fm_predicton)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


Root Mean Squared Error (RMSE) on test data = 9439.28


Clearly LinearRegression peforms better than FMRegressor